In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

matplotlib.rc("axes", unicode_minus = False)

In [3]:
df1 = pd.read_csv("/home/piai/바탕화면/반도체 공정 프로젝트/B1_반도체/01 Oxidation.csv")
df2 = pd.read_csv("/home/piai/바탕화면/반도체 공정 프로젝트/B1_반도체/02 Photo_softbake.csv")
df3 = pd.read_csv("/home/piai/바탕화면/반도체 공정 프로젝트/B1_반도체/03 Photo_lithograpy.csv")
df4 = pd.read_csv("/home/piai/바탕화면/반도체 공정 프로젝트/B1_반도체/04 Etching.csv")
df5 = pd.read_csv("/home/piai/바탕화면/반도체 공정 프로젝트/B1_반도체/05 Ion_Implantation.csv")
df6 = pd.read_csv("/home/piai/바탕화면/반도체 공정 프로젝트/B1_반도체/06 Inspect.csv")

In [4]:
df = pd.merge(df1, df2, how='left', on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df3, how='left', on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df4, how='left', on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df5, how='left', on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df6, how='left', on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])

In [5]:
df.dropna(subset="Thin F2", inplace=True) # 다른 변수들과의 연관성을 찾지못함, 한 행에 여러 열들 결측값 가짐
# pd.set_option('display.max_row', 200)
# pd.set_option('display.max_columns', 200)

In [6]:
df['Ox_Chamber'] = df['Ox_Chamber'].astype('str')
df['photo_soft_Chamber'] = df['photo_soft_Chamber'].astype('str')
df['lithography_Chamber'] = df['lithography_Chamber'].astype('str')
df['Etching_Chamber'] = df['Etching_Chamber'].astype('str')
df['Chamber_Num'] = df['Chamber_Num'].astype('str')
df['path'] = df['Ox_Chamber']+df['photo_soft_Chamber']+df['lithography_Chamber']+df['Etching_Chamber']+df['Chamber_Num']

In [7]:
df.drop(df[df['Oxid_time']<0].index, inplace=True) # 산화시간이 음수
df.drop(df[df['Target']==0].index, inplace=True) # target 값이 0
df.drop(columns = ["Wafer_map","Error_message"],inplace=True) # 웨이퍼맵, 에러메시지

In [8]:
# # 불필요한 열 제거 
# df.drop(columns=["No_Die", "Lot_Num", "Wafer_Num"], inplace=True)

In [9]:
# 시간 데이터 datetime 유형으로 변환
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y')

# 195이상 = 불량(1), 195미만 = 양품(0)
df.loc[df['Target'] >= 195, '불량_195이상'] =1
df.loc[df['Target'] < 195, '불량_195이상'] =0

In [10]:
df.loc[df['Thin F4']<0, 'Thin F4']=df['Thin F4'].median()
df.loc[df['Flux90s']<0, 'Flux90s']=df['Flux90s'].median()
df.loc[df['Flux160s']<0, 'Flux160s']=df['Flux160s'].median()
df.loc[df['Flux160s']<5, 'Flux160s']=df['Flux160s'].median()

In [11]:
df['Thin F1'].fillna(df['Thin F1'].median(), inplace=True)
df['Thin F3'].fillna(df['Thin F3'].median(), inplace=True)
df['Flux60s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux90s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux480s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux840s'].fillna(df['Flux90s'].median(), inplace=True)

In [12]:
negative_values = df.select_dtypes(include=['number']).lt(0)

# 음수 값을 가진 열 목록 출력
columns_with_negative_values = negative_values.any()
print("Columns with negative values:")
print(columns_with_negative_values[columns_with_negative_values].index.tolist())

# 각 열에서 음수 값을 포함한 행 수 출력
print("\nNumber of rows with negative values in each column:")
print(negative_values.sum())

Columns with negative values:
[]

Number of rows with negative values in each column:
Temp_OXid            0
ppm                  0
Pressure             0
Oxid_time            0
thickness            0
Lot_Num              0
Wafer_Num            0
resist_target        0
N2_HMDS              0
pressure_HMDS        0
temp_HMDS            0
temp_HMDS_bake       0
time_HMDS_bake       0
spin1                0
spin2                0
spin3                0
photoresist_bake     0
temp_softbake        0
time_softbake        0
Line_CD              0
Wavelength           0
Resolution           0
Energy_Exposure      0
Thin F4              0
Thin F3              0
Thin F2              0
Thin F1              0
Temp_Etching         0
Source_Power         0
Selectivity          0
Flux60s              0
Flux90s              0
Flux160s             0
Flux480s             0
Flux840s             0
input_Energy         0
Temp_implantation    0
Furance_Temp         0
RTA_Temp             0
Target           

In [14]:
df.head()

,No_Die,Ox_Chamber,process,type,Temp_OXid,Vapor,ppm,Pressure,Oxid_time,thickness,...,Flux160s,Flux480s,Flux840s,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp,Target,path,불량_195이상
0,NOLSM325131325,2,Oxidation,dry,1214.307948,O2,26.69,0.203,120,713.132,...,6.470000e+17,3.010000e+17,6.000000e+17,30795.934,103.433,854.0,154,141.0,21111,0.0
1,NOLSM326132326,2,Oxidation,dry,978.110792,O2,30.86,0.217,137,714.295,...,1.040000e+18,3.030000e+17,6.000000e+17,32135.618,105.552,895.0,156,55.0,21122,0.0
2,NOLSM327133327,2,Oxidation,dry,1176.019149,O2,31.07,0.218,116,710.114,...,6.470000e+17,2.980000e+17,6.000000e+17,31057.895,102.598,898.0,152,96.0,21133,0.0
3,NOLSM328134328,2,Oxidation,dry,933.442733,O2,31.04,0.196,143,710.617,...,3.410000e+17,3.000000e+17,6.000000e+17,32140.566,102.963,879.0,155,105.0,21111,0.0
4,NOLSM329135329,2,Oxidation,wet,1140.598752,H2O,31.37,0.215,76,711.632,...,7.260000e+17,3.020000e+17,6.000000e+17,31986.009,101.419,882.0,155,79.0,21122,0.0


## 최적 path 도출, Target수 그대로 계산

In [21]:
# 'path' 열의 값의 출현 빈도 계산
path_counts = df['path'].value_counts()

# 출현 빈도가 1 이상인 경우만 선택하여 새로운 데이터프레임 생성
df_p = path_counts[path_counts > 1].reset_index()
df_p.columns = ['path', 'count']

# 새로운 데이터프레임 출력
df_p[df_p['path']=='11122']

,path,count
55,11122,12


In [29]:
grouped = df.groupby('path')['Target'].sum().reset_index()
grouped.columns = ['path', 'Target_sum']

# 'path' 값이 동일한 수에 2000을 곱한 값을 계산
grouped['path_count_times_2000'] = grouped['path'].map(df['path'].value_counts()) * 2000

# 새로운 데이터프레임을 생성
df_p = grouped

# 결과 출력
print(df_p)

     path  Target_sum  path_count_times_2000
0   11111      2703.0                  58000
1   11122       988.0                  24000
2   11133      2885.0                  60000
3   11211      1439.0                  24000
4   11222      3861.0                  82000
..    ...         ...                    ...
76  33222      2525.0                  68000
77  33233       812.0                  18000
78  33311      2301.0                  52000
79  33322       601.0                  18000
80  33333      2436.0                  50000

[81 rows x 3 columns]


In [30]:
df_p.head()

,path,Target_sum,path_count_times_2000
0,11111,2703.0,58000
1,11122,988.0,24000
2,11133,2885.0,60000
3,11211,1439.0,24000
4,11222,3861.0,82000


In [32]:
# 전달된 조합 리스트 생성
combinations = [
   ('11111', '22222', '33333'),('11111', '22233', '33322'),
('11111', '22322', '33233'),('11111', '22333', '33222'),
('11111', '23222', '32333'),('11111', '23233', '32322'),
('11111', '23322', '32233'),('11111', '23333', '32222'),
('11122', '22211', '33333'),('11122', '22233', '33311'),
('11122', '22311', '33233'),('11122', '22333', '33211'),
('11122', '23211', '32333'),('11122', '23233', '32311'),
('11122', '23311', '32233'),('11122', '23333', '32211'),
('11133', '22211', '33322'),('11133', '22222', '33311'),
('11133', '22311', '33222'),('11133', '22322', '33211'),
('11133', '23211', '32322'),('11133', '23222', '32311'),
('11133', '23311', '32222'),('11133', '23322', '32211'),
('11211', '22122', '33333'),('11211', '22133', '33322'),
('11211', '22322', '33133'),('11211', '22333', '33122'),
('11211', '23122', '32333'),('11211', '23133', '32322'),
('11211', '23322', '32133'),('11211', '23333', '32122'),
('11222', '22111', '33333'),('11222', '22133', '33311'),
('11222', '22311', '33133'),('11222', '22333', '33111'),
('11222', '23111', '32333'),('11222', '23133', '32311'),
('11222', '23311', '32133'),('11222', '23333', '32111'),
('11233', '22111', '33322'),('11233', '22122', '33311'),
('11233', '22311', '33122'),('11233', '22322', '33111'),
('11233', '23111', '32322'),('11233', '23122', '32311'),
('11233', '23311', '32122'),('11233', '23322', '32111'),
('11311', '22122', '33233'),('11311', '22133', '33222'),
('11311', '22222', '33133'),('11311', '22233', '33122'),
('11311', '23122', '32233'),('11311', '23133', '32222'),
('11311', '23222', '32133'),('11311', '23233', '32122'),
('11322', '22111', '33233'),('11322', '22133', '33211'),
('11322', '22211', '33133'),('11322', '22233', '33111'),
('11322', '23111', '32233'),('11322', '23133', '32211'),
('11322', '23211', '32133'),('11322', '23233', '32111'),
('11333', '22111', '33222'),('11333', '22122', '33211'),
('11333', '22211', '33122'),('11333', '22222', '33111'),
('11333', '23111', '32222'),('11333', '23122', '32211'),
('11333', '23211', '32122'),('11333', '23222', '32111'),
('12111', '21222', '33333'),('12111', '21233', '33322'),
('12111', '21322', '33233'),('12111', '21333', '33222'),
('12111', '23222', '31333'),('12111', '23233', '31322'),
('12111', '23322', '31233'),('12111', '23333', '31222'),
('12122', '21211', '33333'),('12122', '21233', '33311'),
('12122', '21311', '33233'),('12122', '21333', '33211'),
('12122', '23211', '31333'),('12122', '23233', '31311'),
('12122', '23311', '31233'),('12122', '23333', '31211'),
('12133', '21211', '33322'),('12133', '21222', '33311'),
('12133', '21311', '33222'),('12133', '21322', '33211'),
('12133', '23211', '31322'),('12133', '23222', '31311'),
('12133', '23311', '31222'),('12133', '23322', '31211'),
('12211', '21122', '33333'),('12211', '21133', '33322'),
('12211', '21322', '33133'),('12211', '21333', '33122'),
('12211', '23122', '31333'),('12211', '23133', '31322'),
('12211', '23322', '31133'),('12211', '23333', '31122'),
('12222', '21111', '33333'),('12222', '21133', '33311'),
('12222', '21311', '33133'),('12222', '21333', '33111'),
('12222', '23111', '31333'),('12222', '23133', '31311'),
('12222', '23311', '31133'),('12222', '23333', '31111'),
('12233', '21111', '33322'),('12233', '21122', '33311'),
('12233', '21311', '33122'),('12233', '21322', '33111'),
('12233', '23111', '31322'),('12233', '23122', '31311'),
('12233', '23311', '31122'),('12233', '23322', '31111'),
('12311', '21122', '33233'),('12311', '21133', '33222'),
('12311', '21222', '33133'),('12311', '21233', '33122'),
('12311', '23122', '31233'),('12311', '23133', '31222'),
('12311', '23222', '31133'),('12311', '23233', '31122'),
('12322', '21111', '33233'),('12322', '21133', '33211'),
('12322', '21211', '33133'),('12322', '21233', '33111'),
('12322', '23111', '31233'),('12322', '23133', '31211'),
('12322', '23211', '31133'),('12322', '23233', '31111'),
('12333', '21111', '33222'),('12333', '21122', '33211'),
('12333', '21211', '33122'),('12333', '21222', '33111'),
('12333', '23111', '31222'),('12333', '23122', '31211'),
('12333', '23211', '31122'),('12333', '23222', '31111'),
('13111', '21222', '32333'),('13111', '21233', '32322'),
('13111', '21322', '32233'),('13111', '21333', '32222'),
('13111', '22222', '31333'),('13111', '22233', '31322'),
('13111', '22322', '31233'),('13111', '22333', '31222'),
('13122', '21211', '32333'),('13122', '21233', '32311'),
('13122', '21311', '32233'),('13122', '21333', '32211'),
('13122', '22211', '31333'),('13122', '22233', '31311'),
('13122', '22311', '31233'),('13122', '22333', '31211'),
('13133', '21211', '32322'),('13133', '21222', '32311'),
('13133', '21311', '32222'),('13133', '21322', '32211'),
('13133', '22211', '31322'),('13133', '22222', '31311'),
('13133', '22311', '31222'),('13133', '22322', '31211'),
('13211', '21122', '32333'),('13211', '21133', '32322'),
('13211', '21322', '32133'),('13211', '21333', '32122'),
('13211', '22122', '31333'),('13211', '22133', '31322'),
('13211', '22322', '31133'),('13211', '22333', '31122'),
('13222', '21111', '32333'),('13222', '21133', '32311'),
('13222', '21311', '32133'),('13222', '21333', '32111'),
('13222', '22111', '31333'),('13222', '22133', '31311'),
('13222', '22311', '31133'),('13222', '22333', '31111'),
('13233', '21111', '32322'),('13233', '21122', '32311'),
('13233', '21311', '32122'),('13233', '21322', '32111'),
('13233', '22111', '31322'),('13233', '22122', '31311'),
('13233', '22311', '31122'),('13233', '22322', '31111'),
('13311', '21122', '32233'),('13311', '21133', '32222'),
('13311', '21222', '32133'),('13311', '21233', '32122'),
('13311', '22122', '31233'),('13311', '22133', '31222'),
('13311', '22222', '31133'),('13311', '22233', '31122'),
('13322', '21111', '32233'),('13322', '21133', '32211'),
('13322', '21211', '32133'),('13322', '21233', '32111'),
('13322', '22111', '31233'),('13322', '22133', '31211'),
('13322', '22211', '31133'),('13322', '22233', '31111'),
('13333', '21111', '32222'),('13333', '21122', '32211'),
('13333', '21211', '32122'),('13333', '21222', '32111'),
('13333', '22111', '31222'),('13333', '22122', '31211'),
('13333', '22211', '31122'),('13333', '22222', '31111'),]


# 조합에 해당하는 불량품의 총합과 총 칩의 수 계산
def calculate_summary(combinations, df):
    summary = []
    for combo in combinations:
        target_sum = df.loc[df['path'].isin(combo), 'Target_sum'].sum()
        total_chips = df.loc[df['path'].isin(combo), 'path_count_times_2000'].sum()
        defect_ratio = target_sum / total_chips if total_chips > 0 else np.nan
        summary.append({'Combo': combo, 'Total_Defects': target_sum, 'Total_Chips': total_chips, 'Defect_Ratio': defect_ratio})
    return pd.DataFrame(summary)

# 계산 실행
summary_df = calculate_summary(combinations, df_p)

# 불량품 비율로 정렬
sorted_summary_df = summary_df.sort_values(by='Defect_Ratio')

# 결과 출력
sorted_summary_df.head()  # 상위 5개 결과만 출력


,Combo,Total_Defects,Total_Chips,Defect_Ratio
114,"(12233, 21311, 33122)",3723.0,100000,0.037230
115,"(12233, 21322, 33111)",3803.0,98000,0.038806
88,"(12133, 21211, 33322)",3948.0,96000,0.041125
90,"(12133, 21311, 33222)",7524.0,182000,0.041341
186,"(13233, 21311, 32122)",4158.0,100000,0.041580


In [33]:
sorted_summary_df.tail()

,Combo,Total_Defects,Total_Chips,Defect_Ratio
141,"(12333, 23122, 31211)",5814.0,94000,0.061851
159,"(13122, 22333, 31211)",5911.0,94000,0.062883
28,"(11211, 23122, 32333)",4906.0,78000,0.062897
205,"(13322, 22133, 31211)",5671.0,90000,0.063011
133,"(12322, 23133, 31211)",5428.0,82000,0.066195


In [41]:
df_p[(df_p['path'] == '12333') | (df_p['path'] == '23122') | (df_p['path'] == '31211')]

,path,Target_sum,path_count_times_2000
17,12333,3462.0,58000
46,23122,858.0,18000
57,31211,1494.0,18000


## Target 수 195개 이상은 2000으로 카운트

In [46]:
df.loc[df['Target'] >= 195, 'Target'] = 2000

# 결과 출력
df[df['Target']==2000]

,No_Die,Ox_Chamber,process,type,Temp_OXid,Vapor,ppm,Pressure,Oxid_time,thickness,...,Flux160s,Flux480s,Flux840s,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp,Target,path,불량_195이상
16,NOLSM3411317341,2,Oxidation,dry,937.540249,O2,35.73,0.211,94,708.346,...,3.650000e+17,3.030000e+17,6.000000e+17,32773.392,105.126,896.0,150,2000.0,21222,1.0
43,NOLSM3411317,2,Oxidation,dry,919.168661,O2,35.73,0.211,94,708.346,...,3.650000e+17,3.030000e+17,6.000000e+17,32773.392,105.126,896.0,150,2000.0,23133,1.0
59,NOLSM357146357,2,Oxidation,wet,1210.039779,H2O,33.81,0.193,160,713.608,...,8.120000e+17,3.000000e+17,6.000000e+17,31743.858,104.180,914.0,156,2000.0,22133,1.0
77,NOLSM3751424375,2,Oxidation,dry,1275.211613,O2,23.70,0.216,156,715.142,...,4.500000e+17,2.990000e+17,6.000000e+17,32267.154,104.109,907.0,155,2000.0,22333,1.0
86,NOLSM357146,1,Oxidation,wet,1205.923968,H2O,36.97,0.193,160,706.977,...,8.120000e+17,3.000000e+17,6.000000e+17,31743.858,104.180,914.0,156,2000.0,12333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,NOLSM271111,3,Oxidation,dry,1165.554842,O2,40.23,0.194,195,709.964,...,8.390000e+17,3.020000e+17,6.000000e+17,32089.235,100.754,888.0,156,2000.0,32333,1.0
1619,NOLSM8373127,2,Oxidation,wet,1270.900465,H2O,44.80,0.219,26,711.993,...,3.460000e+17,2.990000e+17,6.000000e+17,32255.649,102.317,868.0,153,2000.0,23133,1.0
1646,NOLSM3241227324,1,Oxidation,dry,1095.384157,O2,31.47,0.213,45,715.404,...,6.970000e+17,3.000000e+17,6.000000e+17,31228.026,102.651,887.0,155,2000.0,13333,1.0
1678,NOLSM3141217,2,Oxidation,dry,1229.097573,O2,41.41,0.212,136,702.521,...,8.010000e+17,3.000000e+17,6.000000e+17,30483.424,102.248,893.0,156,2000.0,23111,1.0


In [47]:
grouped = df.groupby('path')['Target'].sum().reset_index()
grouped.columns = ['path', 'Target_sum']

# 'path' 값이 동일한 수에 2000을 곱한 값을 계산
grouped['path_count_times_2000'] = grouped['path'].map(df['path'].value_counts()) * 2000

# 새로운 데이터프레임을 생성
df_p2000 = grouped

# 결과 출력
print(df_p2000)

     path  Target_sum  path_count_times_2000
0   11111      4499.0                  58000
1   11122       988.0                  24000
2   11133      6455.0                  60000
3   11211      4993.0                  24000
4   11222      7438.0                  82000
..    ...         ...                    ...
76  33222      4308.0                  68000
77  33233       812.0                  18000
78  33311      5884.0                  52000
79  33322       601.0                  18000
80  33333      6006.0                  50000

[81 rows x 3 columns]


In [51]:
summary_df2000 = calculate_summary(combinations, df_p2000)

# 불량품 비율로 정렬
sorted_summary_df2000 = summary_df2000.sort_values(by='Defect_Ratio')

# 결과 출력
sorted_summary_df2000.head()  # 상위 5개 결과만 출력

,Combo,Total_Defects,Total_Chips,Defect_Ratio
115,"(12233, 21322, 33111)",3803.0,98000,0.038806
14,"(11122, 23311, 32233)",4080.0,94000,0.043404
55,"(11311, 23233, 32122)",4341.0,96000,0.045219
52,"(11311, 23122, 32233)",4431.0,96000,0.046156
169,"(13211, 21133, 32322)",4675.0,100000,0.046750


In [53]:
sorted_summary_df2000.tail()

,Combo,Total_Defects,Total_Chips,Defect_Ratio
25,"(11211, 22133, 33322)",17449.0,90000,0.193878
141,"(12333, 23122, 31211)",19481.0,94000,0.207245
150,"(13111, 22322, 31233)",19568.0,94000,0.208170
132,"(12322, 23111, 31233)",19651.0,94000,0.209053
205,"(13322, 22133, 31211)",19336.0,90000,0.214844


In [54]:
df_p2000[(df_p2000['path'] == '12333') | (df_p2000['path'] == '23122') | (df_p2000['path'] == '31211')]

,path,Target_sum,path_count_times_2000
17,12333,12167.0,58000
46,23122,858.0,18000
57,31211,6456.0,18000
